In [1]:
import sympy as sp

# Sympy

The python equivalent of the symbolic math toolbox is `sympy`.

## Creating Symbols

### Question 1

Compare a symbolic 1 / 3 to the floating point representation.

In [2]:
third_symbol = sp.Rational(1, 3)
third_symbol

1/3

We can see the difference in floating point precision by asking for 30 decimals of precision from each objects. The binary representation of 1/3 is accurate to about 17 places, while the symbolic representation has arbitrary accuracy.

In [3]:
print(f'Floating point arithmetic:\t {1 / 3:<10.30f}')
print(f'Symbolic arithmetic:\t\t {third_symbol.evalf(30)}')

Floating point arithmetic:	 0.333333333333333314829616256247
Symbolic arithmetic:		 0.333333333333333333333333333333


### Question 2

Compute $sin(\pi)$ symbolically and numerically.

Using floating-point arithmetic, the result is equal to zero to 16 decimal places of accuracy. Using the symbolic computation, we get exactly 0.

In [4]:
import numpy as np
print('Floating point arithmetic:\t', np.sin(np.pi))
print('Symbolic arithmetic:\t\t', sp.sin(sp.pi).evalf(16))

Floating point arithmetic:	 1.2246467991473532e-16
Symbolic arithmetic:		 0


### Question 3

Create multiple variables in one command. For sympy, this is `sp.symbols`

In [5]:
x, y, z = sp.symbols('x y z')

### Question 4
Create 10 indexed variables, $a_1 \dots a_{10}$ and save them in a vector `A`.

We can use a list comprehension together with `sp.symbols` to get everything

In [6]:
a_symbols = sp.symbols([f'a_{i}' for i in range(1, 21)])
A = sp.Matrix(a_symbols).T
A

Matrix([[a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11, a_12, a_13, a_14, a_15, a_16, a_17, a_18, a_19, a_20]])

### Question 5

This question is unique to Matlab. We can create symbolic variables as needed using sp.symbols and tuple unpacking. Sometimes, though, we want to work with all the individual symbols and need them in the global namespace. For this we can use this following hack:

In [7]:
# Jupyter stores all variables in a dictionary called globals,
# with string keys and python object values.
# We update this dictionary to include each individal element of a_symbols

globals().update({var.name:var for var in a_symbols})

In [8]:
# Now we can directly compute with the a symbols
a_1 + a_2 + a_3

a_1 + a_2 + a_3

### Question 6

Create a symbolic matrix

In [9]:
a, b, c = sp.symbols('a b c')
A = sp.Matrix([[a, b, c], [c, a, b], [b, c, a]])
A

Matrix([
[a, b, c],
[c, a, b],
[b, c, a]])

Check that the sum of the elements of the first row equations the sum of the elements of the second column

In [10]:
sum(A[0, :]) == sum(A[:, 1])

True

### Question 7

Make a symbolic matrix with indices and shape, but no data.

In [11]:
A = sp.IndexedBase('A', shape=(2, 4))

# sp.Idx is the special index symbol.
i, j = sp.symbols('i j', cls=sp.Idx)

A[i, j]

A[i, j]

The shape of a symbolic matrix can also be defined via symbols, in this case sp.Integer objects. The shapes are given to the index variables to define their range

In [12]:
n, m = sp.symbols('n m', integer=True)
i, j = [sp.Idx(x, y) for x, y in zip(['i', 'j'], [n, m])]
B = sp.IndexedBase('B', dims=(n, m))

In [13]:
B[i, j]

B[i, j]

## Symbolic Computation

### Question 8

Using `sp.solve`. All symbolic expressions are assumed to equal zero. If you give a symbolic expression to `sp.solve`, it will try to algebraically solve for the requested variables(s).

The function returns a list of solutions. In the following example, there are two solutions for `x`, so it returns a list of two elements.

In [14]:
a, b, c, x = sp.symbols('a b c x')
f = a * x ** 2 + b * x + c
for answer in sp.solve(f, x):
    display(answer)

(-b - sqrt(-4*a*c + b**2))/(2*a)

(-b + sqrt(-4*a*c + b**2))/(2*a)

For the matlab `f == 2` notation, we can use an `sp.Eq` expression, or just subtract 2 (remember that everything is set equal to zero, so we'd be setting it equal to 2 then moving the 2 to the other side)

In [15]:
for answer in sp.solve(sp.Eq(f, 2), x):
    display(answer)

(-b - sqrt(-4*a*c + 8*a + b**2))/(2*a)

(-b + sqrt(-4*a*c + 8*a + b**2))/(2*a)

Unlike matlab, we can't define functions like `g(x) = a * x ** 2 + b * x +  c`. You can define abstract functions and do computation with them, though

In [16]:
g = sp.Function('g')(x)
g - f

-a*x**2 - b*x - c + g(x)

### Question 9

Differentiation is done with the `.diff` method on all sympy objects. Matrices also have a `.jacobian` method.

In [17]:
f = a * x **2 + b * x + c
display(f)
# partial derivatives
display(f.diff(x))
display(f.diff(a))
display(f.diff(b))
display(f.diff(c))

# higher order derivative 
display(f.diff(x, a))

# jacobian
display(sp.Matrix([f]).jacobian([a, b, c, x]))

# We could also just use a list comprehension
# (nested list to make a row vector)
display(sp.Matrix([[f.diff(var) for var in [a, b, c, x]]]))

a*x**2 + b*x + c

2*a*x + b

x**2

x

1

2*x

Matrix([[x**2, x, 1, 2*a*x + b]])

Matrix([[x**2, x, 1, 2*a*x + b]])

### Integration

Integrate using `sp.int`

In [18]:
a, b, c, d, x = sp.symbols('a b c d x')
f = a *x ** 2 + b * x + c * d * x

# by default it does indefinite integration
sp.integrate(f, x)

a*x**3/3 + x**2*(b/2 + c*d/2)

In [19]:
# for definite integration, pass a tuple of (integrand, a, b)
sp.integrate(f, (x, -2, 1))

3*a - 3*b/2 - 3*c*d/2

## Symbolic Simplification

### Question 11

Simplify the following expression using the `.simplify` method

In [20]:
phi = (1 + sp.sqrt(5)) / 2
golden_ratio = phi ** 2 - phi - 1
golden_ratio

-3/2 - sqrt(5)/2 + (1/2 + sqrt(5)/2)**2

In [21]:
golden_ratio.simplify()

0

### Question 12

Expand all terms using the `.expand` method

In [22]:
a = (x ** 2 - 1)
b = (x ** 4 + x ** 3 + x ** 2 + x + 1)
c = (x ** 4 - x ** 3 + x ** 2 - x + 1)
f = a * b * c
f

(x**2 - 1)*(x**4 - x**3 + x**2 - x + 1)*(x**4 + x**3 + x**2 + x + 1)

In [23]:
f.expand()

x**10 - 1

Note that in this case, `.expand` and `.simplify` give the same results

In [24]:
f.simplify()

x**10 - 1

### Question 13

Use `.factor` to get polynomial roots

In [25]:
g = x ** 3 + 6 *  x ** 2 + 11 * x + 6
g.factor()

(x + 1)*(x + 2)*(x + 3)

### Question 14

Get the Horner representation of a polynomial. This is a nested form, which is often efficient for numerical evaluation.

In [26]:
h = x ** 5 + x ** 4 + x ** 3 + x ** 2 + x
h

x**5 + x**4 + x**3 + x**2 + x

In [27]:
sp.horner(h)

x*(x*(x*(x*(x + 1) + 1) + 1) + 1)

### Question 15

Substitute one variable for another using the `.subs` method.

In [28]:
y = sp.Symbol('y')
f = x ** 2 * sp.log(y) + 5 * x * sp.sqrt(y)
f

x**2*log(y) + 5*x*sqrt(y)

In [29]:
# Subs can take a dictionary of target-replacement key-value pairs
# This method is NOT recursive, however.
f.subs({x:3})

15*sqrt(y) + 9*log(y)

In [30]:
# You can also pass a list of tuples, of the form [(target, replacement)]
# This *IS* recursive, because the subs are applied one at a time
f.subs([(x, y)])

5*y**(3/2) + y**2*log(y)

All sympy objects have a `subs` method, including Matrices

In [31]:
a, b, c, alpha, beta = sp.symbols('a b c alpha beta')
A = sp.Matrix([[a, b, c], [c, a, b], [c, b, a]])
A

Matrix([
[a, b, c],
[c, a, b],
[c, b, a]])

In [32]:
# Direct assignment -- remember python is zero indexed!
A[1, 0] = beta
A

Matrix([
[   a, b, c],
[beta, a, b],
[   c, b, a]])

In [33]:
# By default subs are *NOT* done in-place!
display(A.subs({b:alpha}))
display(A)

Matrix([
[   a, alpha,     c],
[beta,     a, alpha],
[   c, alpha,     a]])

Matrix([
[   a, b, c],
[beta, a, b],
[   c, b, a]])

In [34]:
# Sub and save the result
B = A.subs({b:alpha})
B

Matrix([
[   a, alpha,     c],
[beta,     a, alpha],
[   c, alpha,     a]])

### Printing expressions to code

Sympy offers a very powerful library of code writers, allowing compilation of symbolic expressions to many programming languages, including python, C, Rust, Julia, R, and Octave. It also supports popular computational backend packages, including Tensorflow, Pytorch, and Aesara (formerly Theano)

In [35]:
variables = sp.symbols('x y')
params = sp.symbols('alpha beta')
eq1 = x ** alpha * sp.log(y) + beta * x * sp.sqrt(y)
eq2 = x ** 2 - y + beta / alpha * sp.sqrt(x ** 3) * (y / 2)
eq3 = x ** alpha + beta * x
eq4 = sp.log(y) + beta * sp.sqrt(y)
F = sp.Matrix([eq1, eq2, eq3, eq4])
F_jac = F.jacobian([x ,y])
F_jac

Matrix([
[alpha*x**alpha*log(y)/x + beta*sqrt(y), beta*x/(2*sqrt(y)) + x**alpha/y],
[ 2*x + 3*beta*y*sqrt(x**3)/(4*alpha*x),  -1 + beta*sqrt(x**3)/(2*alpha)],
[               alpha*x**alpha/x + beta,                               0],
[                                     0,          beta/(2*sqrt(y)) + 1/y]])

For basic python, we just call `sp.lambdify`. By default, it compiles to vectorized numpy code.

In [36]:
f_jac_np = sp.lambdify(variables + params, F_jac)

In [37]:
f_jac_np(x = 3, y = 2, alpha=0.33, beta = 1)

array([[ 1.52377763,  1.779149  ],
       [13.87295822,  6.87295822],
       [ 1.15806754,  0.        ],
       [ 0.        ,  0.85355339]])

If we wanted to hop into matlab, we could use a code printer, in this case for Octave

In [38]:
octave_code = sp.printing.octave.octave_code(F_jac, assign_to='df')
# octave_code = octave_code
print(octave_code)

df = [alpha.*x.^alpha.*log(y)./x + beta.*sqrt(y) beta.*x./(2*sqrt(y)) + x.^alpha./y; 2*x + 3*beta.*y.*sqrt(x.^3)./(4*alpha.*x) -1 + beta.*sqrt(x.^3)./(2*alpha); alpha.*x.^alpha./x + beta 0; 0 beta./(2*sqrt(y)) + 1./y];


### Question 18

Compare the performance of `lambify` and `subs`

In [39]:
with np.errstate(all='ignore'):
    t_np = %timeit -o f_jac_np(*np.random.normal(size=2), *np.random.uniform(size=2))

7.11 µs ± 64.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [40]:
def make_random_sub_dict():
    xy = np.random.normal(size=2)
    ab = np.random.uniform(size=2)
    args = np.r_[xy, ab]
    return dict(zip([x, y, alpha, beta], args))
t_sp = %timeit -o F_jac.subs(make_random_sub_dict())

6.94 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can go even faster by `jit` compiling `f_jac_np`

In [41]:
import numba as nb
f_jac_jit = nb.njit(f_jac_np)

# Run it once to trigger the compilation
f_jac_jit(*np.random.normal(size=2), *np.random.uniform(size=2));

In [42]:
t_jit = %timeit -o f_jac_jit(*np.random.normal(size=2), *np.random.uniform(size=2))

3.31 µs ± 29.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Speed Test Results
To the surprise of nobody, substitution is the slowest, and JIT-compilation is the fastest.

In [43]:
print(f'{"Package":<10}{"Time (ms)":<12}{"Speedup over Sympy"}')
print('-'*40)
for name, time in zip(['Sympy', 'Numpy', 'Numba'], [t_sp, t_np, t_jit]):
    print(f'{name:<10}{time.average * 1000:<12.5f}{t_sp.average / time.average:0.2f}x')

Package   Time (ms)   Speedup over Sympy
----------------------------------------
Sympy     6.93574     1.00x
Numpy     0.00711     975.40x
Numba     0.00331     2093.33x
